# General operator class
Operator object contain a string identifying its operation.
Want to have $\hat{O}\times\hat{Q} = \hat{O}\hat{Q}$

In [49]:
class Operator:
    def __init__(self):
        self.op = 'U'
        self.unit = 1
        self.im = False

    def __mul__(self,other):
        self.op += other.op
        return self

    def __str__(self):
        if self.op == '':
            return 'I'
        else:
            return self.op
    
    def __invert__(self): # Assume unitary
        return self

## Special operators

### Identity operator
Have
$$
\hat{I} = \begin{bmatrix} 1 & 0 \\ 0 & 1 \end{bmatrix}
$$
Where $\hat{U}\times\hat{I} = \hat{I}\times\hat{U} = \hat{U}$. Solve this by setting it to empty string.

In [52]:
class Identity(Operator):
    def __init__(self):
        self.op = ''

In [53]:
U = Operator()
I = Identity()
print('UI =',U*I)
U = Operator()
I = Identity()
print('IU =',I*U)
I = Identity()
I2 = Identity()
print('II =',I*I2)

UI = U
IU = U
II = I


### Pauli operators
Have the Pauli operators $\sigma_x$,$\sigma_y$ and $\sigma_z$ with
$$
\sigma_x = \begin{bmatrix} 0 & 1 \\ 1 & 0 \end{bmatrix}
,\quad
\sigma_y = \begin{bmatrix} 0 & -i \\ i & 0 \end{bmatrix}
,\quad
\sigma_z = \begin{bmatrix} 1 & 0 \\ 0 & -1 \end{bmatrix}
$$

Want to have unitarity, so that
$$
    \sigma_n^\dagger = \sigma_n \rightarrow \sigma_n^2 = \hat{I}
$$

Furthermore we want 
$$
\sigma_x\sigma_y = i\sigma_z \quad \sigma_y\sigma_x = -i\sigma_z
\\
\sigma_y\sigma_z = i\sigma_x \quad \sigma_z\sigma_y = -i\sigma_x
\\
\sigma_z\sigma_x = i\sigma_y \quad \sigma_x\sigma_z = -i\sigma_y
$$
Note that by setting $x=0$, $y=1$ and $z=2$ the new operator from the above identities is $new = n - n'$.


In [47]:
class Pauli(Operator):
    def __init__(self,operation):
        self.op = operation.upper()
        self.axis = ['X','Y','Z']
        self.ind = self.axis.index(self.op)
        self.unit = 1
        self.im = False
        
    def __mul__(self,other):
        if other.op == self.op:
            self.op = ''
            return 'I'
        elif other.op != self.op and other.op in self.axis:
            op_ind = 3 - (self.ind + other.ind)
            self.op = self.axis[op_ind]
            self.im = not self.im
            if 3*self.ind + other.ind not in [1,5,6]:
                self.unit *= -1
            return self
        else:
            self.op += other.op
            return self
        
    def __str__(self):
        if self.op == '':
            return 'I'
        else:
            return '{}{}{}'.format('-' if self.unit==-1 else '',
                                   'i' if self.im else '',
                                   self.op)

In [44]:
X = Pauli('x')
Y = Pauli('y')
Z = Pauli('z')

print(X,'inverted ->', ~X)
print('X^2 =',(X*~X))

print('')
print(Y,'inverted ->', ~Y)
print('Y^2 =',(Y*Y))

print('')
print(Z,'inverted ->', ~Z)
print('Z^2 =',(Z*Z))

X = Pauli('x')
Y = Pauli('y')
Z = Pauli('z')
print('XY =',X*Y,' , YX =',Y*X)
X = Pauli('x')
Y = Pauli('y')
Z = Pauli('z')
print('YZ =',Y*Z,' , ZY =',Z*Y)
X = Pauli('x')
Y = Pauli('y')
Z = Pauli('z')
print('ZX =',Z*X,' , XZ =',X*Z)

X inverted -> X
X^2 = I

Y inverted -> Y
Y^2 = I

Z inverted -> Z
Z^2 = I
XY = iZ  , YX = -iZ
YZ = iX  , ZY = -iX
ZX = iY  , XZ = -iY


### Raise and lower operator
Have the qubit raising and lowering operators defined as
$$
\sigma_+ = \begin{bmatrix} 0 & 0 \\ 1 & 0 \end{bmatrix} = \sigma_x + i\sigma_y,\quad \sigma_- = \begin{bmatrix} 0 & 1 \\ 0 & 0 \end{bmatrix} = \sigma_x - i\sigma_y
$$
with the actions 
$$
\sigma_+|0> = |1>, \quad \sigma_+|1> = 0
\\
\sigma_-|0> = 0, \quad \sigma_-|1> = |0>
$$
and the identities

$$
\sigma_+^\dagger = \sigma_-, \quad \sigma_-^\dagger = \sigma_+
\\
\sigma_+^2 = 0, \quad \sigma_-^2 = 0
\\
\sigma_+ \sigma_- = \frac{1}{2}(\hat{I} -\sigma_z), \quad \sigma_- \sigma_+ = \frac{1}{2}(\hat{I} +\sigma_z)
\\
\sigma_+\sigma_x = \frac{1}{2}(\hat{I} -\sigma_z), \quad \sigma_x\sigma_+ = \frac{1}{2}(\hat{I} +\sigma_z), \quad \sigma_-\sigma_x = \frac{1}{2}(\hat{I} +\sigma_z), \quad \sigma_x\sigma_- = \frac{1}{2}(\hat{I} -\sigma_z)
\\
\sigma_+\sigma_y = -\frac{i}{2}(\hat{I}-\sigma_z), \quad \sigma_y\sigma_+ = -\frac{i}{2}(\hat{I}+\sigma_z), \quad \sigma_-\sigma_y = \frac{i}{2}(\hat{I}+\sigma_z), \quad \sigma_y\sigma_- = \frac{i}{2}(\hat{I}-\sigma_z)
\\
\sigma_+\sigma_z = \sigma_+, \quad \sigma_z\sigma_+ = -\sigma_+, \quad \sigma_-\sigma_z = -\sigma_-, \quad \sigma_z\sigma_- = \sigma_-
$$

In [ ]:
class Ladder(Operator):
    def __init__(self,operation):
        self.op = operation
        self.values = ['+','-']
        self.ind = self.values.index(self.op)
        
    def __mul__(self,other):
        if other.op == self.op:
            self.op = '0'
        return self
    
    def __invert__(self): 
        self.ind = (self.ind + 1)%2
        self.op = self.values[self.ind]
        return self.op